<a href="https://colab.research.google.com/github/emrealtinok/trading_assistant/blob/main/TradingAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Reads the data

df = pd.read_csv('drive/My Drive/GBPAUD_M15.csv',
                 sep='\t',
                 header=0,
                 names=['datetime', 'open', 'high', 'low', 'close', 'volume']
                 )

# Removes infs and fills empty cells

df = df.replace([np.inf, -np.inf], np.nan)
df.fillna(method='bfill', inplace=True)

# Sets the index
df.set_index("datetime", inplace=True, drop=True)

print(df)

                        open     high      low    close  volume
datetime                                                       
2013-01-23 20:00:00  1.50137  1.50181  1.50130  1.50157     648
2013-01-23 20:15:00  1.50159  1.50223  1.50156  1.50204     567
2013-01-23 20:30:00  1.50202  1.50239  1.50195  1.50223     436
2013-01-23 20:45:00  1.50223  1.50224  1.50157  1.50171     539
2013-01-23 21:00:00  1.50167  1.50205  1.50154  1.50201     538
...                      ...      ...      ...      ...     ...
2021-02-04 12:45:00  1.79066  1.79281  1.78991  1.79243    2544
2021-02-04 13:00:00  1.79243  1.79362  1.79227  1.79333    3417
2021-02-04 13:15:00  1.79338  1.79407  1.79239  1.79385    3085
2021-02-04 13:30:00  1.79387  1.79637  1.79368  1.79620    3070
2021-02-04 13:45:00  1.79626  1.79698  1.79531  1.79544    3110

[200000 rows x 5 columns]


In [3]:
# Price Change
df['price_change'] = df['close'].diff()
df['price_change_percentage'] = df['close'].pct_change()

# Typical Price
df['typical_price'] = (df['close'] + df['high'] + df['low']) / 3
df['typical_price_change'] = df.typical_price.diff()
df['typical_price_change_percentage'] = df['typical_price'].pct_change()

# Simple Moving Average (short)
window_sma1 = 12
df['sma1'] = df.typical_price.rolling(window=window_sma1, min_periods=1).mean()

# Simple Moving Average (long)
window_sma2 = 24
df['sma2'] = df.typical_price.rolling(window=window_sma2, min_periods=1).mean()

# EMAs
ema1 = df.typical_price.ewm(span=12, adjust=False).mean()
ema2 = df.typical_price.ewm(span=24, adjust=False).mean()

# SMA Cross 
sma_diff = df['sma1'] - df['sma2']
sma_diff[sma_diff > 0] = 1
sma_diff[sma_diff < 0] = -1
sma_diff = sma_diff.diff()
sma_diff[sma_diff == -2] = -1
sma_diff[sma_diff == 2] = 1
sma_diff[sma_diff == 0] = 0
df['sma_cross'] = sma_diff

# Standard Deviations
std = df.typical_price.rolling(window=window_sma1, min_periods=1).std()
std2 = df.typical_price.rolling(window=window_sma2, min_periods=1).std()

# Bollinger Bands
df['bb_upper_band1'] = df['sma1'] + 2 * std
df['bb_lower_band1'] = df['sma1'] - 2 * std
df['bb_band_diff1'] = df['bb_upper_band1'] - df['bb_lower_band1']
df['bb_price_diff1'] = df['bb_upper_band1'] - df['typical_price']

df['bb_upper_band2'] = df['sma2'] + 2 * std2
df['bb_lower_band2'] = df['sma2'] - 2 * std2
df['bb_band_diff2'] = df['bb_upper_band2'] - df['bb_lower_band2']
df['bb_price_diff2'] = df['bb_upper_band2'] - df['typical_price']

# Keltner Channels
previous_close = df.close.shift(1)
true_range1 = df.high - df.low
true_range2 = df.high - previous_close
true_range3 = previous_close - df.low
true_range = pd.concat([true_range1, true_range2, true_range3], axis=1).max(axis=1)
df['average_true_range'] = true_range.ewm(span=window_sma1, adjust=False).mean()
df['kc_upper_band'] = ema1 + 2 * df['average_true_range']
df['kc_lower_band'] = ema1 - 2 * df['average_true_range']

# Ichimoku Kinko Hyo

# Tenkan-sen
nine_period_high = df['high'].rolling(window=12, min_periods=1).max()
nine_period_low = df['low'].rolling(window=12, min_periods=1).min()
df['tenkan_sen'] = (nine_period_high + nine_period_low) / 2

# Kijun-sen (Base Line)
period26_high = df.high.rolling(window=24, min_periods=1).max()
period26_low = df.low.rolling(window=24, min_periods=1).min()
df['kijun_sen'] = (period26_high + period26_low) / 2

# Senkou Span A (Leading Span A)
df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(24)

# Senkou Span B (Leading Span B)
period52_high = df.high.rolling(window=48, min_periods=1).max()
period52_low = df.low.rolling(window=48, min_periods=1).min()
df['senkou_span_b'] = ((period52_high + period52_low) / 2).shift(24)

# Kumo (The Cloud)
df['kumo'] = df['senkou_span_a'] - df['senkou_span_b']

# Chikou Span
df['chikou_span'] = df.close.shift(-24)

# RSI
window_rsi = 12
gain = df.typical_price_change.copy()
gain[gain < 0] = 0
loss = df.typical_price_change.copy()
loss[loss > 0] = 0
loss = abs(loss)
avg_gain = gain.rolling(window_rsi, min_periods=1).mean()
avg_loss = loss.rolling(window_rsi, min_periods=1).mean()
df['rsi'] = 100 - (100 / (1 + (avg_gain / avg_loss)))

# Stochastic Oscillator
window_stoc = 12
lowest_low_so = df.low.rolling(window=window_stoc, min_periods=1).min()
highest_high_so = df.high.rolling(window=window_stoc, min_periods=1).max()
df['stochastic_%K'] = 100 * ((df['close'] - lowest_low_so) / (highest_high_so - lowest_low_so))
df['stochastic_%D'] = df['stochastic_%K'].rolling(window=3, min_periods=1).mean()

# MACD
df['macd'] = ema1 - ema2
df['macd_signal'] = df.macd.ewm(span=6, adjust=False).mean()
df['macd_diff'] = df['macd'] - df['macd_signal']

# Money Flow Index
typical_price_change_direction = np.where(df['typical_price_change'] >= 0, 1, -1)
df['raw_money_flow'] = typical_price_change_direction * df['typical_price'] * df['volume']
positive_money_flow = df['raw_money_flow'].copy()
positive_money_flow[positive_money_flow < 0] = 0
negative_money_flow = df['raw_money_flow'].copy()
negative_money_flow[negative_money_flow > 0] = 0
positive_money_flow_14 = positive_money_flow.rolling(window=12, min_periods=1).sum()
negative_money_flow_14 = negative_money_flow.rolling(window=12, min_periods=1).sum().abs()
money_flow_ratio = positive_money_flow_14 / negative_money_flow_14
df['money_flow_index'] = 100 - (100 / (1 + money_flow_ratio))

# Chaikin Oscillator 
chaikin_n = ((df.close - df.low) - (df.high - df.close)) / (df.high - df.low)
chaikin_m = chaikin_n * df.volume
chaikin_adl = chaikin_m.shift(1) + chaikin_m
chaikin_ema1 = chaikin_adl.ewm(span=3, adjust=False).mean()
chaikin_ema2 = chaikin_adl.ewm(span=10, adjust=False).mean()
df['chaikin_osc'] = chaikin_ema1 - chaikin_ema2

# Know Sure Thing
roc_10 = df.typical_price.pct_change(periods=10)
roc_15 = df.typical_price.pct_change(periods=15)
roc_20 = df.typical_price.pct_change(periods=20)
roc_30 = df.typical_price.pct_change(periods=30)
rcma_1 = roc_10.rolling(window=10, min_periods=1).mean()
rcma_2 = roc_15.rolling(window=10, min_periods=1).mean()
rcma_3 = roc_20.rolling(window=10, min_periods=1).mean()
rcma_4 = roc_30.rolling(window=15, min_periods=1).mean()
df['know_sure_thing'] = (rcma_1) + (rcma_2 * 2) + (rcma_3 * 3) + (rcma_4 * 4)
kst_signal = df['know_sure_thing'].rolling(window=9, min_periods=1).mean()
df['know_sure_thing_diff'] = df['know_sure_thing'] - kst_signal

# Balance of Power
bop = (df.close - df.open) / (df.high - df.low)
df['balance_of_power'] = bop.rolling(window=9, min_periods=1).mean()

# Ultimate Oscillator
true_range_max = pd.concat([df.high, previous_close], axis=1).max(axis=1)
true_range_min = pd.concat([df.low, previous_close], axis=1).min(axis=1)
true_range_ult = true_range_max - true_range_min
min_low_pc = pd.concat([df.low, previous_close], axis=1).min(axis=1)
buying_pressure = df.close - min_low_pc
avg_1 = buying_pressure.rolling(window=6, min_periods=1).sum() / true_range_ult.rolling(window=6, min_periods=1).sum()
avg_2 = buying_pressure.rolling(window=12, min_periods=1).sum() / true_range_ult.rolling(window=12, min_periods=1).sum()
avg_3 = buying_pressure.rolling(window=24, min_periods=1).sum() / true_range_ult.rolling(window=24, min_periods=1).sum()
df['ultimate_osc'] = 100 * ((avg_1 * 4 + avg_2 * 2 + avg_3) / 7)

# Average Directional Index

plus_dm = df.high - df.high.shift(1)
minus_dm = df.low.shift(1) - df.low
df['plus_dm'] = np.where((plus_dm > minus_dm) & (plus_dm > 0), plus_dm, 0)
df['minus_dm'] = np.where((minus_dm > plus_dm) & (minus_dm > 0), minus_dm, 0)
plus_dm_smooth = plus_dm.ewm(span=12, adjust=False).mean()
minus_dm_smooth = minus_dm.ewm(span=12, adjust=False).mean()
true_range_smooth = true_range.ewm(span=12, adjust=False).mean()
plus_di = (plus_dm_smooth / true_range_smooth) * 100
minus_di = (minus_dm_smooth / true_range_smooth) * 100
dx = ((plus_di - minus_di).abs()) / ((plus_di + minus_di).abs()) * 100
df['adx'] = dx.ewm(span=14, adjust=False).mean()

# Next Price (for Y values)

df['next_price'] = df['typical_price'].shift(-1)
next_diff = df['next_price'] - df['typical_price']
df['up_or_down'] = np.where(next_diff > 0, 1, 0)

In [4]:
# Defines X and Y
df = df.replace([np.inf, -np.inf], np.nan)
df.fillna(method='bfill', inplace=True)

X = df[['typical_price', 'money_flow_index', 'rsi', 'typical_price_change_percentage', 
        'macd_diff', 'sma_cross', 'stochastic_%K', 'stochastic_%D', 'adx',
        'kc_upper_band', 'kc_lower_band', 'kumo', 'chaikin_osc', 'ultimate_osc',
        'know_sure_thing', 'know_sure_thing_diff', 'balance_of_power',
        'bb_upper_band1', 'bb_lower_band1', 'bb_upper_band2', 'bb_lower_band2',
        'bb_band_diff1', 'bb_band_diff2', 'bb_price_diff1', 'bb_price_diff2',
        ]]
       
Y = df[['up_or_down']]

# Normalizes X
input_scaler = MinMaxScaler()
input_scaler.fit(X)
X = input_scaler.transform(X)

# Converts Y to a NumPy array
Y = Y.to_numpy()

print(X.shape)
print(Y.shape)

(200000, 25)
(200000, 1)


In [5]:
# Prepares the training data for LSTM

X_LSTM = []
for i in range(99, X.shape[0]):
  X_LSTM.append(X[(i - 99):(i + 1)])
X_LSTM = np.array(X_LSTM)
print(X_LSTM.shape)

Y_LSTM = []
for i in range(99, Y.shape[0]):
  Y_LSTM.append(Y[i])
Y_LSTM = np.array(Y_LSTM)
print(Y_LSTM.shape)

(199901, 100, 25)
(199901, 1)


In [6]:
# Builds and runs the neural network

early_stopping = tf.keras.callbacks.EarlyStopping(patience=3,
                                                  monitor="val_loss", 
                                                  restore_best_weights=True)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint('drive/My Drive/trading_assistant',
                                                      save_best_only=True)


model = tf.keras.Sequential([tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.LSTM(50, return_sequences=True),
                             tf.keras.layers.LSTM(50),
                             tf.keras.layers.Dense(50),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Activation('relu'),                       
                             tf.keras.layers.Dense(1, activation='sigmoid')
                             ])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['binary_accuracy']
              )

model.fit(X_LSTM, Y_LSTM,
          batch_size=64,
          epochs=100,
          callbacks=[early_stopping],
          validation_split=0.1,
          )

Epoch 1/100
2812/2812 [==============================] - 44s 13ms/step - loss: 0.6821 - binary_accuracy: 0.5507 - val_loss: 0.7221 - val_binary_accuracy: 0.5933
Epoch 2/100
2812/2812 [==============================] - 36s 13ms/step - loss: 0.6509 - binary_accuracy: 0.6215 - val_loss: 0.6501 - val_binary_accuracy: 0.6212
Epoch 3/100
2812/2812 [==============================] - 36s 13ms/step - loss: 0.6504 - binary_accuracy: 0.6232 - val_loss: 0.6609 - val_binary_accuracy: 0.6099
Epoch 4/100
2812/2812 [==============================] - 35s 13ms/step - loss: 0.6484 - binary_accuracy: 0.6246 - val_loss: 0.6611 - val_binary_accuracy: 0.6097
Epoch 5/100
2812/2812 [==============================] - 35s 12ms/step - loss: 0.6495 - binary_accuracy: 0.6243 - val_loss: 0.6485 - val_binary_accuracy: 0.6238
Epoch 6/100
2812/2812 [==============================] - 35s 12ms/step - loss: 0.6466 - binary_accuracy: 0.6269 - val_loss: 0.6501 - val_binary_accuracy: 0.6249
Epoch 7/100
2812/2812 [===========

In [7]:
X_predict = X_LSTM[-1, :, :]
X_predict = np.expand_dims(X_predict, axis=0)
prediction = model.predict(X_predict)
print(np.where(prediction[0] > 0.50, 'Bullish', 'Bearish'))

['Bearish']
